In [1]:
import pandas as pd
import numpy as np
from scipy.signal import find_peaks
import os
import matplotlib.pyplot as plt
import glob
import shutil # Para eliminar directorios

def extract_normal_mode_windows_optimized(file_path, tao):
    """
    Extrae ventanas de tiempo donde los péndulos oscilan en un modo normal,
    con la condición de no tener picos extra en P2 y P3 entre picos de P1.
    Retorna la duración total de las ventanas encontradas y los datos/metadatos.

    Args:
        file_path (str): Ruta al archivo de datos TSV.
        tao (float): El ancho de la ventana temporal (en segundos) alrededor del pico
                     del péndulo 1 para buscar picos en los otros péndulos.

    Returns:
        tuple: (total_window_duration, list_of_windows_data, list_of_windows_metadata)
               donde list_of_windows_data es una lista de DataFrames y list_of_windows_metadata
               contiene (ti, tf, output_filename, plot_filename, duration) para cada ventana.
               Retorna (0, [], []) si no se encontraron ventanas.
    """
    
    try:
        df = pd.read_csv(file_path, sep='\t', header=None, names=['time', 'pendulum1', 'pendulum2', 'pendulum3'])
    except FileNotFoundError:
        print(f"Error: El archivo '{file_path}' no fue encontrado.")
        return 0, [], []

    time = df['time'].values
    pendulum1 = df['pendulum1'].values
    pendulum2 = df['pendulum2'].values
    pendulum3 = df['pendulum3'].values

    peaks_p1_max, _ = find_peaks(pendulum1)
    peaks_p1_min, _ = find_peaks(-pendulum1)
    all_peaks_p1_indices = np.sort(np.concatenate((peaks_p1_max, peaks_p1_min)))

    if len(all_peaks_p1_indices) < 2:
        return 0, [], []

    all_peaks_p2_indices_max, _ = find_peaks(pendulum2)
    all_peaks_p2_indices_min, _ = find_peaks(-pendulum2)
    all_peaks_p2_indices = np.sort(np.concatenate((all_peaks_p2_indices_max, all_peaks_p2_indices_min)))

    all_peaks_p3_indices_max, _ = find_peaks(pendulum3)
    all_peaks_p3_indices_min, _ = find_peaks(-pendulum3)
    all_peaks_p3_indices = np.sort(np.concatenate((all_peaks_p3_indices_max, all_peaks_p3_indices_min)))

    total_window_duration = 0
    current_windows_data = []
    current_windows_metadata = []

    window_start_p1_idx = -1
    
    for i in range(len(all_peaks_p1_indices) - 1):
        current_p1_peak_idx = all_peaks_p1_indices[i]
        next_p1_peak_idx = all_peaks_p1_indices[i+1]

        t_current_p1_peak = time[current_p1_peak_idx]
        t_next_p1_peak = time[next_p1_peak_idx]

        # Condición 1: Coincidencia de picos al inicio del intervalo
        t_lower_bound_sync = t_current_p1_peak - tao
        t_upper_bound_sync = t_current_p1_peak + tao

        sync_p2 = (time[all_peaks_p2_indices] >= t_lower_bound_sync) & \
                  (time[all_peaks_p2_indices] <= t_upper_bound_sync)
        sync_p3 = (time[all_peaks_p3_indices] >= t_lower_bound_sync) & \
                  (time[all_peaks_p3_indices] <= t_upper_bound_sync)
        
        coincidence_at_current_peak = np.any(sync_p2) and np.any(sync_p3)

        # Condición 2: No hay picos adicionales en P2 ni P3 entre current_p1_peak_idx y next_p1_peak_idx
        interval_start_time = time[current_p1_peak_idx]
        interval_end_time = time[next_p1_peak_idx]

        peaks_p2_in_interval_mask = (time[all_peaks_p2_indices] > interval_start_time) & \
                                     (time[all_peaks_p2_indices] < interval_end_time)
        has_extra_peaks_p2 = np.any(peaks_p2_in_interval_mask)

        peaks_p3_in_interval_mask = (time[all_peaks_p3_indices] > interval_start_time) & \
                                     (time[all_peaks_p3_indices] < interval_end_time)
        has_extra_peaks_p3 = np.any(peaks_p3_in_interval_mask)

        no_extra_peaks = not has_extra_peaks_p2 and not has_extra_peaks_p3

        is_mode_normal_segment = coincidence_at_current_peak and no_extra_peaks

        if is_mode_normal_segment:
            if window_start_p1_idx == -1:
                window_start_p1_idx = current_p1_peak_idx
            
            if i == len(all_peaks_p1_indices) - 2:
                t_i = time[window_start_p1_idx]
                t_f = t_next_p1_peak
                duration = t_f - t_i
                
                window_end_data_idx = np.where(time <= t_f)[0][-1]
                data_window = df.iloc[window_start_p1_idx:window_end_data_idx+1].copy() # .copy() para evitar SettingWithCopyWarning
                
                total_window_duration += duration
                
                base_name = os.path.basename(file_path)
                file_id = base_name.split('_')[0]
                run_id = base_name.split('_')[1].split('.')[0]
                output_filename = f"{file_id}_{run_id}_{t_i:.3f}_{t_f:.3f}.tsv"
                plot_filename = f"{file_id}_{run_id}_{t_i:.3f}_{t_f:.3f}.png"

                current_windows_data.append(data_window)
                current_windows_metadata.append({
                    'ti': t_i, 'tf': t_f, 'duration': duration,
                    'output_filename': output_filename,
                    'plot_filename': plot_filename,
                    'original_file': file_path,
                    'tao_used': tao
                })
                window_start_p1_idx = -1
        else:
            if window_start_p1_idx != -1:
                t_i = time[window_start_p1_idx]
                t_f = time[current_p1_peak_idx] # El tiempo del pico de P1 donde la condición falló
                duration = t_f - t_i

                window_end_data_idx = np.where(time <= t_f)[0][-1]
                data_window = df.iloc[window_start_p1_idx:window_end_data_idx+1].copy()
                
                total_window_duration += duration

                base_name = os.path.basename(file_path)
                file_id = base_name.split('_')[0]
                run_id = base_name.split('_')[1].split('.')[0]
                output_filename = f"{file_id}_{run_id}_{t_i:.3f}_{t_f:.3f}.tsv"
                plot_filename = f"{file_id}_{run_id}_{t_i:.3f}_{t_f:.3f}.png"

                current_windows_data.append(data_window)
                current_windows_metadata.append({
                    'ti': t_i, 'tf': t_f, 'duration': duration,
                    'output_filename': output_filename,
                    'plot_filename': plot_filename,
                    'original_file': file_path,
                    'tao_used': tao
                })
                window_start_p1_idx = -1
    
    return total_window_duration, current_windows_data, current_windows_metadata

# --- Lógica principal para iterar sobre taos y archivos ---

# Crea un archivo de prueba si no tienes uno
if not os.path.exists("001_16_filtrado.tsv"):
    print("Creando archivo de prueba: 001_16_filtrado.tsv")
    test_data = """0	9.18	0.1	1.62
2.533	-9.72	0.18	0.36
2.584	-9.9	0.0	1.26
2.638	-8.1	-0.36	1.98
2.691	-5.04	-0.72	2.52
2.748	-0.54	-1.08	2.88
2.806	3.96	-1.26	3.06
2.858	7.2	-1.26	2.7
2.913	9.18	-1.44	2.16
2.968	9.18	-1.08	1.62
3.023	7.2	-0.54	1.08
3.076	3.96	0.0	0.54
3.131	-0.54	0.54	0.0
3.186	-5.04	1.08	-0.54
3.241	-8.1	1.26	-1.08
3.296	-9.72	1.44	-1.44
3.350	-9.9	1.44	-1.62
3.405	-9.18	1.26	-1.80
3.460	-7.2	1.08	-1.98
3.515	-3.96	0.72	-2.16
3.570	0.54	0.18	-2.34
3.625	5.04	-0.18	-2.52
3.680	8.1	-0.54	-2.70
3.735	9.72	-0.90	-2.88
3.790	9.9	-1.08	-3.06
3.845	9.18	-1.26	-3.24
3.900	7.2	-1.44	-3.42
3.955	3.96	-1.44	-3.60
4.010	-0.54	-1.26	-3.78
4.065	-5.04	-1.08	-3.96
4.120	-8.1	-0.72	-4.14
4.175	-9.72	-0.36	-4.32
4.230	-9.9	0.00	-4.50
4.285	-9.18	0.18	-4.68
4.340	-7.2	0.36	-4.86
4.395	-3.96	0.54	-5.04
4.450	0.54	0.72	-5.22
4.505	5.04	0.90	-5.40
4.560	8.1	1.08	-5.58
4.615	9.72	1.26	-5.76
4.670	9.9	1.44	-5.94
4.725	9.18	1.44	-6.12
4.780	7.2	1.26	-6.30
4.835	3.96	1.08	-6.48
4.890	-0.54	0.72	-6.66
4.945	-5.04	0.36	-6.84
5.000	-8.1	0.00	-7.02
5.055	-9.72	-0.18	-7.20
5.110	-9.9	-0.36	-7.38
5.165	-9.18	-0.54	-7.56
"""
    with open("001_16_filtrado.tsv", "w") as f:
        f.write(test_data)
    
    print("Creando segundo archivo de prueba: 002_17_filtrado.tsv")
    test_data_2 = """0	5.0	0.2	1.0
0.5	4.5	0.1	0.9
1.0	3.0	0.0	0.8
1.5	1.0	-0.1	0.7
2.0	-1.0	-0.2	0.6
2.5	-3.0	-0.1	0.5
3.0	-4.5	0.0	0.4
3.5	-5.0	0.1	0.3
4.0	-4.5	0.2	0.2
4.5	-3.0	0.1	0.1
5.0	-1.0	0.0	0.0
5.5	1.0	-0.1	-0.1
6.0	3.0	-0.2	-0.2
6.5	4.5	-0.1	-0.3
7.0	5.0	0.0	-0.4
7.5	4.5	0.1	-0.5
8.0	3.0	0.2	-0.6
8.5	1.0	0.1	-0.7
9.0	-1.0	0.0	-0.8
9.5	-3.0	-0.1	-0.9
10.0	-4.5	-0.2	-1.0
10.5	-5.0	-0.1	-0.9
11.0	-4.5	0.0	-0.8
11.5	-3.0	0.1	-0.7
12.0	-1.0	0.2	-0.6
12.5	1.0	0.1	-0.5
13.0	3.0	0.0	-0.4
13.5	4.5	-0.1	-0.3
14.0	5.0	-0.2	-0.2
14.5	4.5	-0.1	-0.1
15.0	3.0	0.0	0.0
"""
    with open("002_17_filtrado.tsv", "w") as f:
        f.write(test_data_2)


# Rango de valores de tao a probar
tolerances_tao = [0.05, 0.1, 0.15, 0.2] 

# Directorios de salida base
output_dir_base = "normal_mode_windows_optimized"
plot_dir_base = "normal_mode_plots_optimized"

# Limpiar directorios anteriores para asegurar que solo se guarde lo óptimo
if os.path.exists(output_dir_base):
    shutil.rmtree(output_dir_base)
os.makedirs(output_dir_base)

if os.path.exists(plot_dir_base):
    shutil.rmtree(plot_dir_base)
os.makedirs(plot_dir_base)

print(f"\nIniciando búsqueda de ventanas de modo normal optimizadas...")

# Lista para almacenar todas las ventanas óptimas encontradas, para luego ordenarlas
all_optimal_windows_summary = []

# Iterar sobre todos los archivos .tsv filtrados en el directorio actual
for file_path in glob.glob("*_filtrado.tsv"):
    print(f"\n--- Procesando archivo: {file_path} ---")
    
    best_tao = None
    max_duration = -1
    best_windows_data = []
    best_windows_metadata = []

    for tao_value in tolerances_tao:
        print(f"  Probando con tao = {tao_value:.2f} s")
        # Aquí se llama a la función que retorna la duración total y los metadatos
        duration, windows_data, windows_metadata = extract_normal_mode_windows_optimized(file_path, tao_value)
        
        print(f"    Duración total para tao={tao_value:.2f}: {duration:.3f} s")
        
        if duration > max_duration:
            max_duration = duration
            best_tao = tao_value
            best_windows_data = windows_data
            best_windows_metadata = windows_metadata # Corregido: Debe ser windows_metadata
    
    if best_tao is not None:
        print(f"\n¡{file_path}: Tao óptimo encontrado = {best_tao:.2f} s con una duración total de {max_duration:.3f} s!")
        
        # Ahora guardamos los archivos y gráficas para el mejor tao
        for window_data, window_meta in zip(best_windows_data, best_windows_metadata):
            output_path = os.path.join(output_dir_base, window_meta['output_filename'])
            plot_path = os.path.join(plot_dir_base, window_meta['plot_filename'])
            
            window_data.to_csv(output_path, sep='\t', index=False, header=False)
            print(f"  Guardado TSV: {window_meta['output_filename']}")
            
            plt.figure(figsize=(12, 6))
            plt.plot(window_data['time'], window_data['pendulum1'], label='Péndulo 1')
            plt.plot(window_data['time'], window_data['pendulum2'], label='Péndulo 2')
            plt.plot(window_data['time'], window_data['pendulum3'], label='Péndulo 3')
            plt.xlabel('Tiempo (s)')
            plt.ylabel('Ángulo (grados)')
            plt.title(f"Modo Normal - {os.path.basename(file_path).replace('_filtrado.tsv', '')} - Tiempos: {window_meta['ti']:.3f}s a {window_meta['tf']:.3f}s (Tao={best_tao:.2f}s)")
            plt.legend()
            plt.grid(True)
            plt.savefig(plot_path)
            plt.close()
            print(f"  Guardada Gráfica: {window_meta['plot_filename']}")

            # Añadir esta ventana a la lista global para el resumen final
            # Aseguramos que 'tao_used' se agregue aquí si no lo fue antes
            window_meta['tao_used_for_this_window'] = best_tao # Agregamos el tao óptimo específico para esta ventana
            all_optimal_windows_summary.append(window_meta)
    else:
        print(f"\n{file_path}: No se encontraron ventanas de modo normal para ningún tao probado.")

print("\nProceso de optimización de ventanas de modo normal completado.")

# Ordenar la lista de ventanas por duración, de mayor a menor
all_optimal_windows_summary.sort(key=lambda x: x['duration'], reverse=True)

if all_optimal_windows_summary:
    print("\n--- Ventanas de Modo Normal (Ordenadas por Duración, Mayor a Menor) ---")
    print("{:<12} {:<18} {:<12} {:<10} {:<10} {:<30}".format("Duración (s)", "Archivo Original", "Tao Usado", "T_Inicio", "T_Fin", "Nombre Archivo Guardado"))
    print("-" * 102) # Ajustar el ancho de la línea de separación
    for window in all_optimal_windows_summary:
        print("{:<12.3f} {:<18} {:<12.2f} {:<10.3f} {:<10.3f} {:<30}".format(
            window['duration'],
            os.path.basename(window['original_file']),
            window['tao_used_for_this_window'], # Usar la clave correcta para el tao
            window['ti'],
            window['tf'],
            window['output_filename']
        ))
    print("-" * 102)
else:
    print("\nNo se encontraron ventanas de modo normal en ningún archivo con los parámetros dados.")


Iniciando búsqueda de ventanas de modo normal optimizadas...

--- Procesando archivo: 010_26_filtrado.tsv ---
  Probando con tao = 0.05 s
    Duración total para tao=0.05: 0.000 s
  Probando con tao = 0.10 s
    Duración total para tao=0.10: 5.171 s
  Probando con tao = 0.15 s
    Duración total para tao=0.15: 7.787 s
  Probando con tao = 0.20 s
    Duración total para tao=0.20: 9.487 s

¡010_26_filtrado.tsv: Tao óptimo encontrado = 0.20 s con una duración total de 9.487 s!
  Guardado TSV: 010_26_38.378_38.925.tsv
  Guardada Gráfica: 010_26_38.378_38.925.png
  Guardado TSV: 010_26_44.371_44.876.tsv
  Guardada Gráfica: 010_26_44.371_44.876.png
  Guardado TSV: 010_26_46.682_47.243.tsv
  Guardada Gráfica: 010_26_46.682_47.243.png
  Guardado TSV: 010_26_49.049_49.645.tsv
  Guardada Gráfica: 010_26_49.049_49.645.png
  Guardado TSV: 010_26_50.294_50.846.tsv
  Guardada Gráfica: 010_26_50.294_50.846.png
  Guardado TSV: 010_26_52.690_53.241.tsv
  Guardada Gráfica: 010_26_52.690_53.241.png
  Gu